In [1]:
# %load prepareStats.py
#!/usr/bin/env python

"""
Don't repeat yourself, they say. So this is the stuff that goes in the beginning of all my 
Jupyter Notebooks exploring various F1 statistics. https://github.com/michalkasparek/f1-stats/
"""

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Let's define some useful lists first

unknownResult = ["nan0n", "NaN", "NA", "inf"]
crashes = ["Accident", "Collision", "Fatal accident", "Collision damage", "Spun off"]

Europe = ["Austria", "Austrian", "Azerbaijan", "Belgian", "Belgium", "British", "Czech", "Danish", "Dutch", "East German", "Finnish", "France", "French", "German", "Germany", "Hungarian", "Hungary", "Irish", "Italian", "Italy", "Liechtensteiner", "Monaco", "Monegasque", "Netherlands", "Polish", "Portugal", "Portuguese", "Russia", "Russian", "Spanish", "Spain", "Sweden", "Swedish", "Swiss", "Switzerland", "Turkey", "UK"]
NAmerica = ["American", "Canada", "Canadian", "Mexican", "Mexico", "USA"]
SAmerica = ["Argentina", "Argentine", "Brazil", "Brazilian", "Chilean", "Colombian", "Uruguayan", "Venezuelan"]
Asia = ["Bahrain", "Chinese", "China", "Hong Kong", "India", "Indian", "Indonesian", "Japan", "Japanese", "Korea", "Malaysia", "Malaysian", "Qatar", "Saudi Arabia", "Singapore", "Thai", "UAE"]
Africa = ["Morocco", "Rhodesian", "South Africa", "South African"]
Oceania = ["Australia", "Australian", "New Zealander"]
multiple = ["American-Italian", "Argentine-Italian"]

streetCircuits = ["Melbourne", "Monte-Carlo", "Montreal", "Valencia", "Marina Bay", "Sochi", "Baku", "Jeddah", "Adelaide", "Phoenix", "Detroit", "Dallas", "Nevada", "California", "Oporto", "Lisbon"]

# Now let's load the stats (source: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

drivers = pd.read_csv(os.path.join("data", "drivers.csv"))
results = pd.read_csv(os.path.join("data", "results.csv"))
races = pd.read_csv(os.path.join("data", "races.csv"))
circuits = pd.read_csv(os.path.join("data", "circuits.csv"))
status = pd.read_csv(os.path.join("data", "status.csv"))
constructors = pd.read_csv(os.path.join("data", "constructors.csv"))

# Merge all the tables into a single dataframe

results = results.merge(drivers, on="driverId", how="right")
races = races.merge(circuits, on="circuitId", how="right")
results = results.merge(races, on="raceId", how="right")
results = results.merge(status, on="statusId", how="right")
results = results.merge(constructors, on="constructorId", how="right")

# Make the columns more useful 

results["fullname"] = results["forename"] + " " + results["surname"]
results["date"] = results["date"].str.slice(0,6) + results["year"].astype(str).str.slice(0,4)
results["date"] = pd.to_datetime(results["date"], format="%d/%m/%Y")
results["year"] = results["year"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
results["round"] = results["round"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
results["position"] = results["position"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
results["fastestLapSpeed"] = results["fastestLapSpeed"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")

# Make some new columns

results["year_round"] = results["year"].astype(str).str.slice(0,4) + results["round"].astype(str).str.slice(0,-2).str.zfill(2)
results = results[~results.year_round.isin(unknownResult)]
results["year_round"] = results["year_round"].apply(pd.to_numeric, errors = "coerce").astype(int)

results.loc[results.nationality_x.isin(Europe),"driverContinent"]="Europe"
results.loc[results.nationality_x.isin(NAmerica),"driverContinent"]="NAmerica"
results.loc[results.nationality_x.isin(SAmerica),"driverContinent"]="SAmerica"
results.loc[results.nationality_x.isin(Asia),"driverContinent"]="Asia"
results.loc[results.nationality_x.isin(Africa),"driverContinent"]="Africa"
results.loc[results.nationality_x.isin(Oceania),"driverContinent"]="Oceania"
results.loc[results.nationality_x.isin(multiple),"driverContinent"]="multiple"

results.loc[results.location.isin(streetCircuits),"street"]=True

entries = pd.Series(results.groupby(["driverId"]).size(), name="entries")
results = results.merge(entries, on = ["driverId"], how = "right")

# Split the main dataframe for wins, podiums and top 6 finishes only 

wins = results[results["position"] == 1]
podiums = results[results["position"] < 4]
top6 = results[results["position"] < 7]

# Finally let's give the plots some swag

plt.style.use('_mpl-gallery')
plt.rcParams["figure.figsize"] = (20,3)

C:\Users\micha\AppData\Local\Temp\ipykernel_7440\2856688637.py:44: FutureWarning: Passing 'suffixes' which cause duplicate columns {'url_x'} in the result is deprecated and will raise a MergeError in a future version.
  results = results.merge(constructors, on="constructorId", how="right")
C:\Users\micha\AppData\Local\Temp\ipykernel_7440\2856688637.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results["year_round"] = results["year_round"].apply(pd.to_numeric, errors = "coerce").astype(int)


In [2]:
totalLocations = pd.Series(results.groupby(["fullname"])["location"].nunique(), name="totalLocations")
results = results.merge(totalLocations, on="fullname", how="right")

In [3]:
driversIndyOnly = results[(results["totalLocations"] == 1) & (results["location"] == "Indianapolis")].fullname.drop_duplicates().tolist()

In [4]:
firstGP = results["year_round"].min()
lastGP = results["year_round"].max()
firstDrivers = results[results["year_round"] == firstGP].fullname.tolist()
lastDrivers = results[results["year_round"] == lastGP].fullname.tolist()

## The shiniest debuts

(Without the drivers participating in the very first F1 race, and without the drivers participating only in Indianapolis 500.)

In [5]:
results = results.sort_values(by="year_round").reset_index()

In [6]:
firstRaces = results.iloc[results.groupby(["driverId"])["year_round"].agg(pd.Series.idxmin)]

firstRaces = firstRaces[~firstRaces.fullname.isin(firstDrivers)]

firstRaces = firstRaces[~firstRaces.fullname.isin(driversIndyOnly)]

firstRaces = firstRaces[["fullname", "year", "location", "position"]].sort_values(by=["year", "position"])

In [7]:
firstRaces[["fullname", "year", "location", "position"]].sort_values(by=["position", "year"]).head(50)

,fullname,year,location,position
2319,Giancarlo Baghetti,1961.0,Reims,1.0
35,Alberto Ascari,1950.0,Monte-Carlo,2.0
135,Dorino Serafini,1950.0,Monza,2.0
893,Karl Kling,1954.0,Reims,2.0
15283,Jacques Villeneuve,1996.0,Melbourne,2.0
22151,Kevin Magnussen,2014.0,Melbourne,2.0
345,Jean Behra,1952.0,Bern,3.0
1083,Cesare Perdisa,1955.0,Monte-Carlo,3.0
1420,Masten Gregory,1957.0,Monte-Carlo,3.0
4384,Reine Wisell,1970.0,New York State,3.0


## The most successful farewells

(Without the drivers participating in the very last race.)

In [8]:
results = results.sort_values(by="year_round", ascending=True).reset_index(drop=True)

In [9]:
lastRaces = results.loc[results.groupby(["fullname"])["year_round"].agg(pd.Series.idxmax)]

In [10]:
lastRaces = lastRaces[~lastRaces.fullname.isin(lastDrivers)]
lastRaces = lastRaces[~lastRaces.fullname.isin(driversIndyOnly)]

In [11]:
lastRaces[["fullname", "year", "location", "position"]].sort_values(by=["position", "year"]).head(20)

,fullname,year,location,position
238,Luigi Fagioli,1951.0,Reims,1.0
3724,Jim Clark,1968.0,Midrand,1.0
151,Dorino Serafini,1950.0,Monza,2.0
1283,Paul Frère,1956.0,Spa,2.0
1789,Mike Hawthorn,1958.0,Casablanca,2.0
4675,Jo Siffert,1971.0,New York State,2.0
14382,Alain Prost,1993.0,Adelaide,2.0
22114,Mark Webber,2013.0,São Paulo,2.0
23366,Nico Rosberg,2016.0,Abu Dhabi,2.0
2430,Tony Brooks,1961.0,New York State,3.0


It's safe to say that not a single driver has ever knowingly ended his F1 career with a satisfying victory. 

Luigi Fagioli's farewell win was a shared drive, [Fagioli finishing his part 11th](https://en.wikipedia.org/wiki/Luigi_Fagioli):

> His only Grand Prix of 1951 was his last, but he nevertheless won the French Grand Prix with Juan-Manuel Fangio, earning the distinction of being the oldest person to ever win a Formula One race. During the race, the Alfa Romeo team manager ordered him to hand over his healthy car to Fangio while Fagioli would drive Fangio's car, which was plagued with engine problems. Ferrari had done the same, ordering José Froilán González to hand over to the quicker and more experienced Alberto Ascari; this was common practice in Grand Prix racing before 1957. Fangio battled hard with Ascari and took victory while Fagioli finished 11th and last in Fangio's original car, 22 laps down. Fagioli was so incensed by this that he retired from Grand Prix racing after this race.

Jim Clark was killed in a F2 race before the 1968 Spanish Grand Prix, little did he know the 1968 South African Grand Prix was his farewell win.

## Careers crowned with the best ever result in the very last race

In [12]:
bestRaces = results.sort_values(by="year_round", ascending=True)

bestRaces = bestRaces.loc[bestRaces.groupby(["fullname"])["position"].agg(pd.Series.idxmin).dropna()]

bestRaces = bestRaces[["fullname", "year", "location", "position"]].sort_values(by=["position", "year"])

bestLastRaces = lastRaces.merge(bestRaces, on="fullname", how="right")

In [13]:
bestLastRaces = bestLastRaces[bestLastRaces["position_x"] == bestLastRaces["position_y"]]

bestLastRaces = bestLastRaces[bestLastRaces["location_x"] == bestLastRaces["location_y"]]

bestLastRaces = bestLastRaces[bestLastRaces["entries"] > 1]

In [14]:
bestLastRaces[["fullname", "location_x", "year_x", "position_x", "entries"]].dropna().sort_values(by="entries", ascending=False).head(10)

,fullname,location_x,year_x,position_x,entries
336,Jan Magnussen,Montreal,1998.0,6.0,25.0
587,Jérôme d'Ambrosio,Monza,2012.0,13.0,20.0
383,Corrado Fabi,Dallas,1984.0,7.0,18.0
204,Michael Andretti,Monza,1993.0,3.0,13.0
123,Paul Frère,Spa,1956.0,2.0,11.0
415,Bruce Halford,Reims,1960.0,8.0,9.0
6,Luigi Fagioli,Reims,1951.0,1.0,8.0
373,Ingo Hoffmann,São Paulo,1977.0,7.0,6.0
626,Skip Barber,New York State,1972.0,16.0,6.0
422,Sam Tingle,Midrand,1969.0,8.0,6.0


## Careers launched with the best result, never to be achieved again 

This is kind of funny: Jan Magnussen ended his career with his best result (6th place), then 14 years later his son Kevin started his own F1 career with his own best result (2nd place). Both have the longest careers among the drivers with similar fates.

In [15]:
bestRaces = results.sort_values(by="year_round", ascending=False)

bestRaces = bestRaces.loc[bestRaces.groupby(["fullname"])["position"].agg(pd.Series.idxmin).dropna()]

bestRaces = bestRaces[["fullname", "year", "location", "position", "entries"]].sort_values(by=["position", "year"])

bestFirstRaces = firstRaces.merge(bestRaces, on="fullname", how="right")

In [16]:
bestFirstRaces = bestFirstRaces[bestFirstRaces["position_x"] == bestFirstRaces["position_y"]]

bestFirstRaces = bestFirstRaces[bestFirstRaces["location_x"] == bestFirstRaces["location_y"]]

bestFirstRaces = bestFirstRaces[bestFirstRaces["entries"] > 1]

In [17]:
bestFirstRaces[["fullname", "location_x", "year_x", "position_x", "entries"]].dropna().sort_values(by="entries", ascending=False).head(10)

,fullname,location_x,year_x,position_x,entries
154,Kevin Magnussen,Melbourne,2014.0,2.0,123
301,Felipe Nasr,Melbourne,2015.0,5.0,40
22,Giancarlo Baghetti,Reims,1961.0,1.0,26
188,Reine Wisell,New York State,1970.0,3.0,23
221,Ken Wharton,Bern,1952.0,4.0,16
190,Mark Donohue,Ontario,1971.0,3.0,15
238,Vic Elford,Rouen,1968.0,4.0,13
117,Karl Kling,Reims,1954.0,2.0,12
602,Ian Ashley,Nürburg,1974.0,14.0,11
263,Alan Brown,Bern,1952.0,5.0,9


## Longest and shortest time in F1 after the last win or podium

(Without the drivers participating in the very last race.)

In [18]:
lastRaces = results.sort_values(by="year_round", ascending=True).reset_index(drop=True)
lastRaces = results.loc[results.groupby(["fullname"])["year_round"].agg(pd.Series.idxmax)]

In [19]:
lastRaces = lastRaces[~lastRaces.fullname.isin(lastDrivers)]
lastRaces = lastRaces[~lastRaces.fullname.isin(driversIndyOnly)]

In [20]:
lastWins = wins.sort_values(by="year_round", ascending=False).reset_index(drop=True)
lastWins = lastWins.loc[lastWins.groupby(["fullname"])["year_round"].agg(pd.Series.idxmax)]

In [21]:
afterLastWin = lastWins.merge(lastRaces, on="fullname", how="right")

In [22]:
afterLastWin["era"] = (afterLastWin["date_y"] - afterLastWin["date_x"])

In [23]:
afterLastWin.groupby(["fullname"]).agg({"era": max}).sort_values(by="era", ascending=False).head(10)

,era
fullname,
Robert Kubica,4844 days
Jo Bonnier,4508 days
Michele Alboreto,3388 days
Felipe Massa,3311 days
Jacques Villeneuve,3227 days
Olivier Panis,3066 days
Troy Ruttman,2922 days
Jarno Trulli,2744 days
Jochen Mass,2646 days


In [24]:
afterLastWin.groupby(["fullname"]).agg({"era": min}).sort_values(by="era", ascending=True).head(10)

,era
fullname,
Luigi Fagioli,0 days
Jim Clark,0 days
Mika Häkkinen,14 days
Peter Collins,15 days
Ronnie Peterson,28 days
Jochen Rindt,35 days
Didier Pironi,36 days
Nico Rosberg,49 days
Jo Siffert,49 days


In [25]:
lastPodiums = podiums.sort_values(by="year_round", ascending=False).reset_index(drop=True)
lastPodiums = lastPodiums.loc[lastPodiums.groupby(["fullname"])["year_round"].agg(pd.Series.idxmax)]

In [26]:
afterLastPodium = lastPodiums.merge(lastRaces, on="fullname", how="right")

In [27]:
afterLastPodium["era"] = (afterLastPodium["date_y"] - afterLastPodium["date_x"])

In [28]:
afterLastPodium.groupby(["fullname"]).agg({"era": max}).sort_values(by="era", ascending=False).head(10)

,era
fullname,
Jo Bonnier,4508 days
Robert Kubica,4032 days
Derek Warwick,3381 days
Jos Verstappen,3332 days
Bruno Giacomelli,3270 days
Rolf Stommelen,2975 days
Troy Ruttman,2922 days
Louis Chiron,2919 days
Olivier Panis,2695 days
